In [1]:
import numpy as np # linear algebra
import pandas as pd
from pathlib import Path
INPUT = Path("../input/google-smartphone-decimeter-challenge/")

In [2]:
df_baseline = pd.read_csv(INPUT/"baseline_locations_train.csv")
df_sample_trail_gt = pd.read_csv(INPUT/"train/2020-05-14-US-MTV-1/Pixel4/ground_truth.csv")
df_sample_trail = pd.read_csv(INPUT/"train/2020-05-14-US-MTV-1/Pixel4/Pixel4_derived.csv")
df_sample_trail["correctedPrM"] = df_sample_trail["rawPrM"] + df_sample_trail["satClkBiasM"] - df_sample_trail["isrbM"] - df_sample_trail["ionoDelayM"] - df_sample_trail["tropoDelayM"] 

In [3]:
def ecef2lla(x, y, z):
    # x, y and z are scalars or vectors in meters
    x = np.array([x]).reshape(np.array([x]).shape[-1], 1)
    y = np.array([y]).reshape(np.array([y]).shape[-1], 1)
    z = np.array([z]).reshape(np.array([z]).shape[-1], 1)

    a=6378137
    a_sq=a**2
    e = 8.181919084261345e-2
    e_sq = 6.69437999014e-3

    f = 1/298.257223563
    b = a*(1-f)

    # calculations:
    r = np.sqrt(x**2 + y**2)
    ep_sq  = (a**2-b**2)/b**2
    ee = (a**2-b**2)
    f = (54*b**2)*(z**2)
    g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
    c = (e_sq**2)*f*r**2/(g**3)
    s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
    p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
    q = np.sqrt(1 + 2*p*e_sq**2)
    r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
    u = np.sqrt((r - e_sq*r_0)**2 + z**2)
    v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
    z_0 = (b**2)*z/(a*v)
    h = u*(1 - b**2/(a*v))
    phi = np.arctan((z + ep_sq*z_0)/r)
    lambd = np.arctan2(y, x)

    return phi*180/np.pi, lambd*180/np.pi, h

In [4]:
epoch_time = 1273529464442
df_sample_epoch = df_sample_trail[df_sample_trail.millisSinceGpsEpoch == epoch_time]
df_sample_epoch_gt = df_sample_trail_gt[df_sample_trail_gt.millisSinceGpsEpoch == epoch_time]
df_sample_epoch_baseline = df_baseline[(df_baseline.collectionName == "2020-05-14-US-MTV-1") & (df_baseline.phoneName == "Pixel4") & (df_baseline.millisSinceGpsEpoch == epoch_time)]

sat_pos = df_sample_epoch[["xSatPosM","ySatPosM","zSatPosM"]].to_numpy()
pseudoranges = np.squeeze(df_sample_epoch[["correctedPrM"]].to_numpy())

In [5]:
def least_squares(sat_pos, pseudoranges, weights=1, x_hat=np.array([0, 0, 0, 0])):
    """
    Args:
    sat_pos: The satellite position (meters) in an ECEF coordinate frame
    pseudoranges: The corrected pseudorange (i.e. a closer approximation to the geometric range from the phone to the satellite)
    x_hat: the phone's initial/previous estimated position (x, y, z, b) and 
           b represent the user clock bias in units of distance = clock bias (t) * light speed (c)

    Returns:
    x_hat: the phone's estimated position
    norm_dp:
    """
    dx = np.Inf*np.ones(3)
    G = np.ones((pseudoranges.size, 4))
    iterations = 0

    if isinstance(weights, np.ndarray):
        weights = np.diag(weights)
    else:
        weights = weights*np.eye(pseudoranges.size)

    while np.linalg.norm(dx) > 1e-3:
        norms = np.linalg.norm(sat_pos - x_hat[:3], axis=1)
        dp = pseudoranges - norms - x_hat[3]
        G[:, 0:3] = -(sat_pos - x_hat[:3])/norms[:, None]
        # G_T = np.transpose(G)
        # dx = np.linalg.inv(G_T@G) @ G_T @ dp
        dx = np.linalg.pinv(weights@G) @ weights @ dp
        x_hat = x_hat + dx
        iterations += 1
    return x_hat, np.linalg.norm(dp)

In [6]:
pseudoranges_sigma = np.squeeze(df_sample_epoch[["rawPrUncM"]].to_numpy())

x, dp = least_squares(sat_pos, pseudoranges, 1/pseudoranges_sigma)

print("Ground truth:", df_sample_epoch_gt[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Weighted Least Square Estimation (Baseline):", df_sample_epoch_baseline[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Weighted Least Square Estimation:", *ecef2lla(*x[:3]))

Ground truth: [[  37.42357595 -122.09413204   33.21      ]]
Weighted Least Square Estimation (Baseline): [[  37.4235777 -122.094101   -33.29     ]]
Weighted Least Square Estimation: [[37.42357917]] [[-122.09375829]] [[-35.25074978]]


In [16]:
import scipy.optimize as opt

def calc_pos_fix(sat_pos, pr, weights=1, x0=[0, 0, 0, 0]):
    '''
    Calculates gps fix with WLS optimizer
    returns:
    0 -> list with positions
    1 -> pseudorange errs
    '''
    n = len(pr)
    if n < 3:
        return x0, []

    Fx_pos = pr_residual(sat_pos, pr, weights=weights)
    opt_pos = opt.least_squares(Fx_pos, x0).x
    return opt_pos, Fx_pos(opt_pos, weights=1)


def pr_residual(sat_pos, pr, weights=1):
    # solve for pos
    def Fx_pos(x_hat, weights=weights):
        rows = weights * (np.linalg.norm(sat_pos - x_hat[:3], axis=1) + x_hat[3] - pr)
        return rows
    return Fx_pos

In [17]:
x, dp = calc_pos_fix(sat_pos, pseudoranges)

print("Ground truth:", df_sample_epoch_gt[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Weighted Least Square Estimation (Baseline):", df_sample_epoch_baseline[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Simple Least Square Estimation:", *ecef2lla(*x[:3]))
# print(x[3]/LIGHTSPEED)
# print(dp)

Ground truth: [[  37.42357595 -122.09413204   33.21      ]]
Weighted Least Square Estimation (Baseline): [[  37.4235777 -122.094101   -33.29     ]]
Simple Least Square Estimation: [[37.42361372]] [[-122.0936959]] [[-26.18615579]]


In [18]:
pseudoranges_sigma = np.squeeze(df_sample_epoch[["rawPrUncM"]].to_numpy())

x, dp = calc_pos_fix(sat_pos, pseudoranges, 1/pseudoranges_sigma)

print("Ground truth:", df_sample_epoch_gt[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Weighted Least Square Estimation (Baseline):", df_sample_epoch_baseline[["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]].to_numpy())
print("Weighted Least Square Estimation:", *ecef2lla(*x[:3]))

Ground truth: [[  37.42357595 -122.09413204   33.21      ]]
Weighted Least Square Estimation (Baseline): [[  37.4235777 -122.094101   -33.29     ]]
Weighted Least Square Estimation: [[37.42357917]] [[-122.09375829]] [[-35.25075022]]


In [19]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [20]:
deg_gt = df_sample_epoch_gt[["latDeg","lngDeg"]].to_numpy().transpose()
deg_baseline = df_sample_epoch_baseline[["latDeg","lngDeg"]].to_numpy().transpose()
print("Weighted Least Square (baseline) haversine distance (M):", calc_haversine(*deg_gt, *deg_baseline))
print("Weighted Least Square haversine distance (M):", calc_haversine(*deg_gt, *ecef2lla(*x[:3])[:2]))

Weighted Least Square (baseline) haversine distance (M): [2.74590055]
Weighted Least Square haversine distance (M): [[32.98570795]]


In [21]:
def simple_pipeline(df_trails):
    """ simple pipeline to estimate the GNSS receiver location by least square
    Args:
    df_trails: the df read from derived file

    Returns:
    result df with estimated degrees and heights
    """
    df_trails["correctedPrM"] = df_trails["rawPrM"] + df_trails["satClkBiasM"] - df_trails["isrbM"] - df_trails["ionoDelayM"] - df_trails["tropoDelayM"]

    results = []
    x = [0, 0, 0, 0]
    df_epochs = df_trails.groupby(["collectionName", "phoneName", "millisSinceGpsEpoch"])
    for indices, df_epoch in tqdm(df_epochs, desc="Estimate location by LS for epoch"):
        sat_pos = df_epoch[["xSatPosM","ySatPosM","zSatPosM"]].to_numpy()
        pseudoranges = np.squeeze(df_epoch[["correctedPrM"]].to_numpy())
        pseudoranges_sigma = np.squeeze(df_epoch[["rawPrUncM"]].to_numpy())
        x, _ = calc_pos_fix(sat_pos, pseudoranges, 1/pseudoranges_sigma, x)
        # x, _ = calc_pos_fix(sat_pos, pseudoranges, 1, x)
        values = np.squeeze(ecef2lla(*x[:3]))
        results.append([*indices, *values])
    return pd.DataFrame(results,columns=["collectionName", "phoneName", "millisSinceGpsEpoch", "latDeg", "lngDeg", "heightAboveWgs84EllipsoidM"])

In [22]:
from pathlib import Path
# from tqdm import tqdm
from tqdm.notebook import tqdm

# INPUT = Path("./data")
ground_truths = (INPUT / "train").rglob("ground_truth.csv")
drived_files = (INPUT / "train").rglob("*_derived.csv")

df_sample_trails_baseline = pd.read_csv(INPUT / 'baseline_locations_train.csv')
df_sample_trails_gt = pd.concat([pd.read_csv(filepath) for filepath in tqdm(ground_truths, total=73, desc="Reading ground truth data")], ignore_index=True)
df_sample_trails = pd.concat([pd.read_csv(filepath) for filepath in tqdm(drived_files, total=73, desc="Reading drived data")], ignore_index=True)

Reading ground truth data:   0%|          | 0/73 [00:00<?, ?it/s]

Reading drived data:   0%|          | 0/73 [00:00<?, ?it/s]

In [23]:
df_sample_trails_gt["receivedSvTimeInGpsNanos"] = df_sample_trails_gt.millisSinceGpsEpoch*int(1e6)
df_sample_trails_raw = df_sample_trails.drop("millisSinceGpsEpoch", axis=1)

df_merge = pd.merge_asof(df_sample_trails_raw.sort_values('receivedSvTimeInGpsNanos'), df_sample_trails_gt.sort_values('receivedSvTimeInGpsNanos'), 
                                           on="receivedSvTimeInGpsNanos", by=["collectionName", "phoneName"], direction='nearest',tolerance=int(1e9))
df_merge = df_merge.sort_values(by=["collectionName", "phoneName", "millisSinceGpsEpoch"], ignore_index=True)

In [24]:
df_sample_trails_estimate = simple_pipeline(df_merge)

Estimate location by LS for epoch:   0%|          | 0/130756 [00:00<?, ?it/s]

In [27]:
df_sample_trails_merged_baseline = pd.merge_asof(df_sample_trails_gt.sort_values('millisSinceGpsEpoch'),
                                                 df_sample_trails_baseline.sort_values('millisSinceGpsEpoch'), 
                                                 on="millisSinceGpsEpoch", by=["collectionName", "phoneName"], 
                                                 direction='nearest',tolerance=100000, suffixes=('_truth', '_pred'))
df_sample_trails_merged_baseline = df_sample_trails_merged_baseline.sort_values(by=["collectionName", "phoneName", "millisSinceGpsEpoch"], ignore_index=True)

df_sample_trails_merged_SL = pd.merge_asof(df_sample_trails_gt.sort_values('millisSinceGpsEpoch'), 
                                           df_sample_trails_estimate.sort_values('millisSinceGpsEpoch'), 
                                           on="millisSinceGpsEpoch", by=["collectionName", "phoneName"], 
                                           direction='nearest',tolerance=100000, suffixes=('_truth', '_pred'))
df_sample_trails_merged_SL = df_sample_trails_merged_SL.sort_values(by=["collectionName", "phoneName", "millisSinceGpsEpoch"], ignore_index=True)

compared_cols = ["latDeg_truth","lngDeg_truth","latDeg_pred","lngDeg_pred"]
print("Weighted Least Square (baseline) haversine distance (M):", calc_haversine(*df_sample_trails_merged_baseline[compared_cols].to_numpy().transpose()).mean())
print("Weighted Least Square haversine distance (M):", calc_haversine(*df_sample_trails_merged_SL[compared_cols].to_numpy().transpose()).mean())

Weighted Least Square (baseline) haversine distance (M): 3.8468483749952074
Weighted Least Square haversine distance (M): 30.854276673073244


## Submission

In [28]:
from pathlib import Path
# from tqdm import tqdm
from tqdm.notebook import tqdm

# INPUT = Path("./data")
drived_files = (INPUT / "test").rglob("*_derived.csv")

df_sample_trails_baseline = pd.read_csv(INPUT / 'baseline_locations_test.csv')
df_sample_trails = pd.concat([pd.read_csv(filepath) for filepath in tqdm(drived_files, total=48, desc="Reading drived data")], ignore_index=True)

Reading drived data:   0%|          | 0/48 [00:00<?, ?it/s]

In [ ]:
df_sample_trails_baseline["receivedSvTimeInGpsNanos"] = df_sample_trails_baseline.millisSinceGpsEpoch*int(1e6)
df_sample_trails_raw = df_sample_trails.drop("millisSinceGpsEpoch", axis=1)

df_merge = pd.merge_asof(df_sample_trails_raw.sort_values('receivedSvTimeInGpsNanos'), df_sample_trails_baseline.sort_values('receivedSvTimeInGpsNanos'), 
                                           on="receivedSvTimeInGpsNanos", by=["collectionName", "phoneName"], direction='nearest',tolerance=int(1e9))
df_merge = df_merge.sort_values(by=["collectionName", "phoneName", "millisSinceGpsEpoch"], ignore_index=True)

In [ ]:
df_sample_trails_estimate = simple_pipeline(df_merge)

In [ ]:
df_sample_trails_baseline = df_sample_trails_baseline.drop(["latDeg","lngDeg","heightAboveWgs84EllipsoidM"], axis=1)
df_sample_trails_merged = pd.merge_asof(df_sample_trails_baseline.sort_values('millisSinceGpsEpoch'), 
                                        df_sample_trails_estimate.sort_values('millisSinceGpsEpoch'), 
                                        on="millisSinceGpsEpoch", by=["collectionName", "phoneName"], direction='nearest', tolerance=100000)
df_sample_trails_merged = df_sample_trails_merged.sort_values(by=["phone", "millisSinceGpsEpoch"], ignore_index=True)

df_submission = df_sample_trails_merged[["phone", "millisSinceGpsEpoch", "latDeg", "lngDeg"]].copy()
df_submission.to_csv('../output/sub_nb010.csv', index=False)